In [15]:
# importing libraries,reading csv into a dataframe
import numpy as np
import pandas as pd
import scipy.stats as stats
data = pd.read_csv("boston.csv")

### EDA

In [16]:
from pandas import set_option
set_option('precision', 2)
data.head(2)

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,6.32e-03,18.0,2.31,0,0.54,6.58,65.2,4.09,1,296,15.3,396.9,4.98,24.0
1,2,2.73e-02,0.0,7.07,0,0.47,6.42,78.9,4.97,2,242,17.8,396.9,9.14,21.6


In [17]:
data.shape

(506, 15)

In [18]:
data=data.drop('Unnamed: 0',1)

In [19]:
data.dtypes

crim       float64
zn         float64
indus      float64
chas         int64
nox        float64
rm         float64
age        float64
dis        float64
rad          int64
tax          int64
ptratio    float64
black      float64
lstat      float64
medv       float64
dtype: object

In [20]:
data.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
count,5.06e+02,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00
mean,3.61e+00,11.36,11.14,0.07,0.55,6.28,68.57,3.80,9.55,408.24,18.46,356.67,12.65,22.53
std,8.60e+00,23.32,6.86,0.25,0.12,0.70,28.15,2.11,8.71,168.54,2.16,91.29,7.14,9.20
min,6.32e-03,0.00,0.46,0.00,0.39,3.56,2.90,1.13,1.00,187.00,12.60,0.32,1.73,5.00
25%,8.20e-02,0.00,5.19,0.00,0.45,5.89,45.02,2.10,4.00,279.00,17.40,375.38,6.95,17.02
50%,2.57e-01,0.00,9.69,0.00,0.54,6.21,77.50,3.21,5.00,330.00,19.05,391.44,11.36,21.20
75%,3.68e+00,12.50,18.10,0.00,0.62,6.62,94.07,5.19,24.00,666.00,20.20,396.23,16.96,25.00
max,8.90e+01,100.00,27.74,1.00,0.87,8.78,100.00,12.13,24.00,711.00,22.00,396.90,37.97,50.00


In [21]:
set_option('precision', 1)
corr = data.corr()
corr.iloc[:,-1:]

,medv
crim,-0.4
zn,0.4
indus,-0.5
chas,0.2
nox,-0.4
rm,0.7
age,-0.4
dis,0.2
rad,-0.4
tax,-0.5


### Random forest model for prediction

In [6]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestRegressor()
X = data.drop('medv',1)
y = data['medv']
cross_val_score(rfc, X, y, cv=10)

C:\Users\MGrover\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\MGrover\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\MGrover\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\MGrover\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 10

array([ 0.67352501,  0.79318097,  0.1387743 ,  0.7679547 ,  0.82543522,
        0.67738301,  0.60550525,  0.33549533, -0.85438214,  0.22120459])

In [7]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

### Linear Regression model 

In [8]:
# Including this to remind you that it is necessary to use numpy arrays rather 
# than lists otherwise you will get an error
X2 = np.array(X)
y2 = np.array(y)

regr = LinearRegression().fit(X2, y2)

cv_results = cross_validate(regr, X, y, cv=3)

### MSE from Regression Model

In [9]:
scores_regr = cross_validate(regr, X2, y2, cv=10,scoring=('r2', 'neg_mean_squared_error'),
                        return_train_score=True)
print(scores_regr['test_neg_mean_squared_error']) 

[  -9.28694671  -14.15128316  -14.07360615  -35.20692433  -31.88511666
  -19.83587796   -9.94726918 -168.37537954  -33.32974507  -10.96041068]


In [11]:
MSE_regr = scores_regr['test_neg_mean_squared_error']
print("Accuracy: %0.2f (+/- %0.2f)" % (MSE_regr.mean(), MSE_regr.std() * 2))

Accuracy: -34.71 (+/- 91.15)


### MSE from Random Forest Model

In [13]:
scores_rfc = cross_validate(rfc, X, y, cv=10,scoring=('r2', 'neg_mean_squared_error'),
                        return_train_score=True)
print(scores_rfc['test_neg_mean_squared_error']) 

C:\Users\MGrover\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\MGrover\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\MGrover\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\MGrover\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 10

[-13.31512745  -5.93787451  -4.7073     -30.52311961 -14.07219412
 -35.25699412  -8.7118     -97.564956   -35.871428   -21.941728  ]


C:\Users\MGrover\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [14]:
MSE_rfc = scores_rfc['test_neg_mean_squared_error']
print("Accuracy: %0.2f (+/- %0.2f)" % (MSE_rfc.mean(), MSE_rfc.std() * 2))

Accuracy: -26.79 (+/- 52.13)


### Conclusion:
#### Performance of Random Forest regressor is much better then Linear regression model
#### Random Forest has a lower MSE and lower standard deviation in predictions
#### Having said that both models have a very poor performance.